In [46]:
import train_wrn_ebm as train
import argparse
from torch_geometric import utils
from torch_geometric.datasets import TUDataset


In [37]:

parser = argparse.ArgumentParser("Energy Based Models and Shit")
parser.add_argument("--dataset", type=str, default="cifar10", choices=["cifar10", "svhn", "cifar100"])
parser.add_argument("--data_root", type=str, default="log_out")

# optimization
parser.add_argument("--lr", type=float, default=1e-4)
parser.add_argument("--decay_epochs", nargs="+", type=int, default=[160, 180],
                    help="decay learning rate by decay_rate at these epochs")
parser.add_argument("--decay_rate", type=float, default=.3,
                    help="learning rate decay multiplier")
parser.add_argument("--clf_only", action="store_true", help="If set, then only train the classifier")
parser.add_argument("--labels_per_class", type=int, default=-1,
                    help="number of labeled examples per class, if zero then use all labels")
parser.add_argument("--optimizer", choices=["adam", "sgd"], default="adam")
parser.add_argument("--batch_size", type=int, default=64)
parser.add_argument("--n_epochs", type=int, default=200)
parser.add_argument("--warmup_iters", type=int, default=-1,
                    help="number of iters to linearly increase learning rate, if -1 then no warmmup")
# loss weighting
parser.add_argument("--p_x_weight", type=float, default=1.)
parser.add_argument("--p_y_given_x_weight", type=float, default=1.)
parser.add_argument("--p_x_y_weight", type=float, default=0.)
# regularization
parser.add_argument("--dropout_rate", type=float, default=0.0)
parser.add_argument("--sigma", type=float, default=3e-2,
                    help="stddev of gaussian noise to add to input, .03 works but .1 is more stable")
parser.add_argument("--weight_decay", type=float, default=0.0)
# network
parser.add_argument("--norm", type=str, default=None, choices=[None, "norm", "batch", "instance", "layer", "act"],
                    help="norm to add to weights, none works fine")
# EBM specific
parser.add_argument("--n_steps", type=int, default=20,
                    help="number of steps of SGLD per iteration, 100 works for short-run, 20 works for PCD")
parser.add_argument("--width", type=int, default=10, help="WRN width parameter")
parser.add_argument("--depth", type=int, default=28, help="WRN depth parameter")
parser.add_argument("--uncond", action="store_true", help="If set, then the EBM is unconditional")
parser.add_argument("--class_cond_p_x_sample", action="store_true",
                    help="If set we sample from p(y)p(x|y), othewise sample from p(x),"
                            "Sample quality higher if set, but classification accuracy better if not.")
parser.add_argument("--buffer_size", type=int, default=10000)
parser.add_argument("--reinit_freq", type=float, default=.05)
parser.add_argument("--sgld_lr", type=float, default=1.0)
parser.add_argument("--sgld_std", type=float, default=1e-2)
# logging + evaluation
parser.add_argument("--save_dir", type=str, default='./experiment')
parser.add_argument("--ckpt_every", type=int, default=10, help="Epochs between checkpoint save")
parser.add_argument("--eval_every", type=int, default=1, help="Epochs between evaluation")
parser.add_argument("--print_every", type=int, default=100, help="Iterations between print")
parser.add_argument("--load_path", type=str, default=None)
parser.add_argument("--print_to_log", action="store_true", help="If true, directs std-out to log file")
parser.add_argument("--plot_cond", action="store_true", help="If set, save class-conditional samples")
parser.add_argument("--plot_uncond", action="store_true", help="If set, save unconditional samples")
parser.add_argument("--n_valid", type=int, default=5000)



_StoreAction(option_strings=['--n_valid'], dest='n_valid', nargs=None, const=None, default=5000, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [47]:
dataset = TUDataset(root = "molecule_dataset", name = "MUTAG")

Processing...
Done!


In [89]:
maxNodes = 0
for i in range(len(dataset)):
    n = dataset[i].num_nodes
    if (n > maxNodes):
        maxNodes = n

print(maxNodes)

28


In [109]:
testMol = dataset[0]
s = testMol.edge_stores[0]
s

{'edge_index': tensor([[ 0,  0,  1,  1,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  6,  6,  7,  7,
          8,  8,  8,  9,  9,  9, 10, 10, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14,
         15, 16],
        [ 1,  5,  0,  2,  1,  3,  2,  4,  9,  3,  5,  6,  0,  4,  4,  7,  6,  8,
          7,  9, 13,  3,  8, 10,  9, 11, 10, 12, 11, 13, 14,  8, 12, 12, 15, 16,
         14, 14]]), 'x': tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0.],
        [0

In [113]:
print(s['edge_index'].size())   # 2 x E edges
print(s['x'].size())            # N nodes x 7 node features
print(s['edge_attr'].size())    #E edges X 4 edge features

torch.Size([2, 38])
torch.Size([17, 7])
torch.Size([38, 4])


GOAL:
1) create ordered map of (edge_type, vector_idx)
2) create function to use map to identify vector location

In [ ]:
vector_map = dict()

pos_ctr = 0
for i in range(28):
    for j in range(i):
        vector_map[(j, i)] = pos_ctr
        pos_ctr += 1

In [ ]:
args = parser.parse_args([])
args.n_classes = 100 if args.dataset == "cifar100" else 10
train.main(args)